In [8]:
import sys, json, datetime

import matplotlib.pyplot as plt
from matplotlib.scale import LogScale
import seaborn as sns
import InfiniQuantumSim.TLtensor as tlt

sns.set_theme(style="whitegrid")

In [9]:
circuit_n_qubits = {
    'W': range(5,35),
    'H': range(5,36),
    'QFT': range(5,25),
    'GHZ': range(5,35),
    'QPE': range(5,19)
} 
contr_methods = ['sqlite', 'psql', 'ducksql']
n_runs = 20

circuits = {
    # 'GHZ': tlt.generate_ghz_circuit, 
    # 'W': tlt.generate_w_circuit,
    'QFT': tlt.generate_qft_circuit
    # 'QPE': tlt.generate_qpe_circuit,
    # 'H': tlt.generate_hadamard_wall
}
results = {}

## Benchmark run for all circuits

In [10]:
mem_limit_bytes = 2**34
tim_limit_seconds = 2**4
for cname, circuit in circuits.items():
    oom = ["psql", "ducksql"]
    results[cname] = {}
    progress = 0
    l_qbits = len(circuit_n_qubits[cname])-1
    for n_qubits in circuit_n_qubits[cname]:
        sys.stdout.write('\r')
        circuit_dict = circuit(n_qubits)
        qc = tlt.QuantumCircuit(circuit_dict=circuit_dict)

        results[cname][n_qubits] = qc.benchmark_ciruit_performance(n_runs, oom = oom)
        for method in results[cname][n_qubits].keys():
               if method in oom or method == "eqc":
                    continue
               mem_avg = sum(results[cname][n_qubits][method]["memory"])/n_runs
               tim_avg = sum(results[cname][n_qubits][method]["time"])/n_runs
               if "sql" in method:
                    mem_avg += sum(results[cname][n_qubits]["eqc"]["tensor"]["memory"])/n_runs
                    tim_avg += sum(results[cname][n_qubits]["eqc"]["tensor"]["time"])/n_runs
                    mem_avg += sum(results[cname][n_qubits]["eqc"]["contraction"]["memory"])/n_runs
                    tim_avg += sum(results[cname][n_qubits]["eqc"]["contraction"]["time"])/n_runs

               if  mem_avg >= mem_limit_bytes:
                    print(f'{method} kicked out due to memory limitations!\n')
                    oom.append(method)
               elif tim_avg >= tim_limit_seconds:
                    print(f'{method} kicked out due to time limitations!\n')
                    oom.append(method)

        sys.stdout.write("[%-20s] %d%%" % ('='*int((20/l_qbits)*progress), progress*(100/l_qbits)))
        sys.stdout.flush()
        progress += 1

    print("\n" + cname + " done!")

sqlite kicked out due to time limitations!

[====================] 100%
QFT done!


### Save Results to json File

In [11]:
fname = datetime.datetime.now().strftime("results/%m%d%y_MBP_QFTonly") + ".json"

with open(fname, "w") as file:
    json.dump(results, file)

## Benchmark run changing circuit depth

In [3]:
circuit_depth_map = {
    n_qubits: range(5, 40) for n_qubits in range(5, 20)
}
contr_methods = ['np-one-shot', 'np-mps', 'sqlite', 'psql', 'ducksql']
n_runs = 20

circuit = tlt.generate_ghz_proned
results = {}

In [ ]:
mem_limit_bytes = 2**34
tim_limit_seconds = 2**3
for n_qubits, depths in circuit_depth_map.items():
    oom = ["ducksql", "psql"]
    results[n_qubits] = {}
    progress = 0
    l_qbits = len(circuit_depth_map[n_qubits])-1
    for depth in depths:
        sys.stdout.write('\r')
        circuit_dict = circuit(n_qubits, depth)
        qc = tlt.QuantumCircuit(circuit_dict=circuit_dict)

        results[n_qubits][depth] = qc.benchmark_ciruit_performance(n_runs, oom = oom)
        for method in results[n_qubits][depth].keys():
               if method in oom or method == "eqc":
                    continue
               mem_avg = sum(results[n_qubits][depth][method]["memory"])/n_runs
               tim_avg = sum(results[n_qubits][depth][method]["time"])/n_runs
               if "sql" in method:
                    mem_avg += sum(results[n_qubits][depth]["eqc"]["tensor"]["memory"])/n_runs
                    tim_avg += sum(results[n_qubits][depth]["eqc"]["tensor"]["time"])/n_runs
                    mem_avg += sum(results[n_qubits][depth]["eqc"]["contraction"]["memory"])/n_runs
                    tim_avg += sum(results[n_qubits][depth]["eqc"]["contraction"]["time"])/n_runs

               if  mem_avg >= mem_limit_bytes:
                    print(f'{method} kicked out due to memory limitations!\n')
                    oom.append(method)
               elif tim_avg >= tim_limit_seconds:
                    print(f'{method} kicked out due to time limitations!\n')
                    oom.append(method)

        sys.stdout.write("[%-20s] %d%%" % ('='*int((20/l_qbits)*progress), progress*(100/l_qbits)))
        sys.stdout.flush()
        progress += 1

    print("\n" + str(n_qubits) + " done!")

In [5]:
fname = datetime.datetime.now().strftime("results/%m%d%y_MBP_depth_nq") + ".json"

with open(fname, "w") as file:
    json.dump(results, file)